# SciStream Fabric Example MICH ---- STAR 
### This exmaple test only the SciStream Protocol itself. According to the basic setups, this version may not need switches.
This notebook will setup a demo testbed for Scistream by utilizing Fabric resources. Will support QUIC implementation between channels. And will evaluate the performance through various streaming protocols (e.g., DASH).

Additional resources:
- [SciStream](https://scistream.github.io)
- [FABRIC Forums](https://learn.fabric-testbed.net/forums/)
- [SciStream Repo](https://github.com/scistream)
- [SciStream YouTube Presentations](https://youtu.be/IWKGjAa_TOA)
- [Topology Graph](https://drive.google.com/file/d/124xh4K15c7tUcaETXmj-JSPKcQLA_oTC/view?usp=sharing)

In [1]:
 import os

# If you are using the FABRIC JupyterHub, the following three evnrionment vars
# were automatically provided when you logged in.
#os.environ['FABRIC_CREDMGR_HOST']='cm.fabric-testbed.net'
#os.environ['FABRIC_ORCHESTRATOR_HOST']='orchestrator.fabric-testbed.net'
#os.environ['FABRIC_TOKEN_LOCATION']=os.environ['HOME']+'/work/fabric_token.json'

# Set your FABRIC PROJECT ID
os.environ['FABRIC_PROJECT_ID']='f4344c37-52e3-4d0c-a354-9311342ec109'

# Bastion IPs
os.environ['FABRIC_BASTION_HOST'] = 'bastion-1.fabric-testbed.net'

# Set your Bastion username and private key
os.environ['FABRIC_BASTION_USERNAME']='cqy78_0038438951'
os.environ['FABRIC_BASTION_KEY_LOCATION']=os.environ['HOME']+'/work/fabric_bastion_key' #'id_rsa_fabric'

# Set the keypair FABRIC will install in your slice. 
os.environ['FABRIC_SLICE_PRIVATE_KEY_FILE']=os.environ['HOME']+'/work/id_rsa'
os.environ['FABRIC_SLICE_PUBLIC_KEY_FILE']=os.environ['HOME']+'/work/id_rsa.pub'

# If your slice private key uses a passphrase, set the passphrase
#from getpass import getpass
#print('Please input private key passphrase. Press enter for no passphrase.')
#os.environ['FABRIC_SLICE_PRIVATE_KEY_PASSPHRASE']=getpass()

## Basic FABRIC Slice Configuration

In [2]:
import json
import traceback
from fabrictestbed_extensions.fablib.fablib import fablib

In [3]:
try:
    available_resources = fablib.get_available_resources()
    print(f"Available Resources: {available_resources}")
except Exception as e:
    print(f"Error: {e}")

Error: Failed to get advertised_topology: Status.FAILURE, (504)
Reason: Gateway Time-out
HTTP response headers: HTTPHeaderDict({'Server': 'nginx/1.21.6', 'Date': 'Thu, 18 Aug 2022 04:04:10 GMT', 'Content-Type': 'text/html', 'Content-Length': '167', 'Connection': 'keep-alive'})
HTTP response body: <html>
<head><title>504 Gateway Time-out</title></head>
<body>
<center><h1>504 Gateway Time-out</h1></center>
<hr><center>nginx/1.21.6</center>
</body>
</html>




## Configure Slice Parameters

This section builds the experiment slice. TO BE CHANGED, according to the code. this setup may change.

Note: Fabric did not support modifying resources after create slides. 



In [3]:
# Slice 
slice_name = 'SciStream-MICH-STAR'

# Switches --- This version may not need any switch
# s_name = "s1" 

# switch_cores = 2
# switch_ram = 8
# switch_disk = 100

# Hosts
h1_name = "h1" # consider as Facility 1 ProdApp
h2_name = "h2" # consider as Facility 1 ProdGN
h3_name = "h3" # consider as S2UC
h4_name = "h4" # consider as Facility 2 ConsGN
h5_name = "h5" # consider as Facility 2 ConsApp

h1_ip="192.168.0.1"
h2_ip="192.168.0.2" # prodGN - command: listener-ip: 192.168.0.2 s2-port 5000
h3_ip="192.168.0.3" # S2UC json file running prod and cons on 5000 but with different ip address
h4_ip="192.168.0.4" # consGN - command: listener-ip: 192.168.0.4 s2-port 5000
h5_ip="192.168.0.5"

h1_LAN_ip="172.16.0.1"
h2_LAN_ip="172.16.0.2"

h2_WAN_ip="172.16.1.1"
h4_WAN_ip="172.16.1.2"

h4_LAN_ip="172.16.2.1"
h5_LAN_ip="172.16.2.2"

host_cores = 32
host_ram = 128
host_disk = 10
# Sites
# Default set to TACC to support IPv4 and github clone...
# Should use proxy to access github once checked as IPv6 address. Or use other git platfrom, e.g., bibucket...
site_1 = 'MICH' # 'SALT' # 'NCSA' # s'WASH' # 'MAX' # 'STAR' # 'NCSA' # 'MAX' # 'UTAH' # 'DALL' # 'TACC' # 'MAX'
site_2 = 'STAR' # 'SALT' # 'NCSA' # s'WASH' # 'MAX' # 'STAR' # 'NCSA' # 'MAX' # 'UTAH' # 'DALL' # 'TACC' # 'MAX'

control_net_name = 'control_net' #no need for S2DS test only
prod_lan_net_name = 'prod_lan_net'
gn_net_name = 'GN_WAN_NET'
cons_lan_net_name = 'cons_lan_net'

net_pub_name = 'open_net'

# All node properties
#username = 'ubuntu'
image = 'default_ubuntu_20'
vlan = '1000'
#image_type = 'qcow2'




### Create the Slice

**This part is optional. Once the slice on, there is no need to have this block running.** \
In terms of the NIC usage. We prefer NIC_ConnectX_6 or NIC_Basic. They both provide 100 Gbps. However, NIC_ConnectX_5 still accepable. \
The network topology will be fixed and non-programmable after creating the slice. Be care of setup l2 or l3 networks. \
Network can be setup cross-site. A vlan is established based on your settings. The RTT is based on the physical distances. \
*In the newest FABRIC release, project manager may need to permissions from Admin to add cross-cite communication. Please attention.*

In [10]:
try:
    #Create Slice
    slice = fablib.new_slice(name=slice_name)
    
    # Add switch node
#     switch1 = slice.add_node(name=s_name, site=site_2)
#     switch1.set_capacities(cores=switch_cores, ram=switch_ram, disk=switch_disk)
#     switch1.set_image(image)
#     #[s1_iface_local] = s1.add_component(model='NIC_Basic', name="s1_local_nic").get_interfaces()
#     [s_iface_to_h1, s_iface_to_h2] = switch1.add_component(model='NIC_ConnectX_6', name="prod_switch_nic").get_interfaces()
#     s_iface_to_h1.set_vlan(vlan=vlan)
#     s_iface_to_h2.set_vlan(vlan=vlan)

#     s_iface_to_h3 = switch1.add_component(model='NIC_Basic', name="uc_local_nic").get_interfaces()[0]
#     s_iface_to_h3.set_vlan(vlan=vlan)
#     [s_iface_to_h4, s_iface_to_h5] = switch1.add_component(model='NIC_ConnectX_6', name="cons_switch_nic").get_interfaces()
#     s_iface_to_h4.set_vlan(vlan=vlan)
#     s_iface_to_h5.set_vlan(vlan=vlan)
    
    # Add host node h1
    h1 = slice.add_node(name=h1_name, site=site_1)
    h1.set_capacities(cores=host_cores, ram=host_ram, disk=host_disk)
    h1.set_image(image)
    h1_lan_iface = h1.add_component(model='NIC_Basic', name="h1_nic").get_interfaces()[0]
    
    # Add host node h2
    h2 = slice.add_node(name=h2_name, site=site_1)
    h2.set_capacities(cores=host_cores, ram=host_ram, disk=host_disk)
    h2.set_image(image)
    h2_lan_iface = h2.add_component(model='NIC_Basic', name="h2_nic").get_interfaces()[0]
    wan_iface1 = h2.add_component(model='NIC_Basic', name="h2_wan_nic").get_interfaces()[0]
 
    # Add host node h3 (no need for S2DS only test, but required for whole SciStream test)
    # h3 = slice.add_node(name=h3_name, site=site_2)
    # h3.set_capacities(cores=host_cores, ram=host_ram, disk=host_disk)
    # h3.set_image(image)
    # h3_iface = h3.add_component(model='NIC_Basic', name="h3_nic").get_interfaces()[0]
    
    # Add host node h4
    h4 = slice.add_node(name=h4_name, site=site_2)
    h4.set_capacities(cores=host_cores, ram=host_ram, disk=host_disk)
    h4.set_image(image)
    h4_lan_iface = h4.add_component(model='NIC_Basic', name="h4_nic").get_interfaces()[0]
    wan_iface2 = h4.add_component(model='NIC_Basic', name="h4_wan_nic").get_interfaces()[0]
    
    # Add host node h5
    h5 = slice.add_node(name=h5_name, site=site_2)
    h5.set_capacities(cores=host_cores, ram=host_ram, disk=host_disk)
    h5.set_image(image)
    h5_lan_iface = h5.add_component(model='NIC_Basic', name="h5_nic").get_interfaces()[0]
    

    #Add control panel networks 
    # control_net = slice.add_l2network(name=control_net_name, interfaces=[h1_iface,h2_iface, h3_iface, h4_iface,h5_iface])
    
    #Add Prod LAN
    prod_lan = slice.add_l2network(name=prod_lan_net_name, interfaces=[h1_lan_iface,h2_lan_iface])
    #Add Cons LAN
    cons_lan = slice.add_l2network(name=cons_lan_net_name, interfaces=[h4_lan_iface,h5_lan_iface])
    
    #Add GN WAN
    gn_wan = slice.add_l2network(name=gn_net_name, interfaces=[wan_iface1,wan_iface2])

    
    #Add L3 network for S2UC (May get error from Fabric)
    #net2 = slice.add_l3network(name=net_pub_name, interfaces=[h3_iface_pub], type='IPv4')
    
    #Submit Slice Request
    slice.submit() #(wait_progress=True)
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()
    

Error: Failed to submit slice: Status.FAILURE, (500)
Reason: INTERNAL SERVER ERROR
HTTP response headers: HTTPHeaderDict({'Server': 'nginx/1.21.6', 'Date': 'Thu, 18 Aug 2022 04:06:59 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Content-Length': '40', 'Connection': 'keep-alive', 'Access-Control-Allow-Credentials': 'true', 'Access-Control-Allow-Headers': 'DNT, User-Agent, X-Requested-With, If-Modified-Since, Cache-Control, Content-Type, Range', 'Access-Control-Allow-Methods': 'GET, POST, PUT, DELETE, OPTIONS', 'Access-Control-Allow-Origin': '*', 'Access-Control-Expose-Headers': 'Content-Length, Content-Range, X-Error', 'X-Error': 'Slice SciStream-MICH-STAR already exists'})
HTTP response body: Slice SciStream-MICH-STAR already exists



Traceback (most recent call last):
  File "/tmp/ipykernel_53/9580073.py", line 69, in <module>
    slice.submit() #(wait_progress=True)
  File "/opt/conda/lib/python3.9/site-packages/fabrictestbed_extensions/fablib/slice.py", line 1258, in submit
    raise Exception("Failed to submit slice: {}, {}".format(return_status, slice_reservations))
Exception: Failed to submit slice: Status.FAILURE, (500)
Reason: INTERNAL SERVER ERROR
HTTP response headers: HTTPHeaderDict({'Server': 'nginx/1.21.6', 'Date': 'Thu, 18 Aug 2022 04:06:59 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Content-Length': '40', 'Connection': 'keep-alive', 'Access-Control-Allow-Credentials': 'true', 'Access-Control-Allow-Headers': 'DNT, User-Agent, X-Requested-With, If-Modified-Since, Cache-Control, Content-Type, Range', 'Access-Control-Allow-Methods': 'GET, POST, PUT, DELETE, OPTIONS', 'Access-Control-Allow-Origin': '*', 'Access-Control-Expose-Headers': 'Content-Length, Content-Range, X-Error', 'X-Error': 'Slice SciS

# timeout needs to be extended

## Get the Slice

In [4]:
# ssh -F ~/work/fabric_config/ssh_config -i ~/work/id_rsa 
try:
    slice = fablib.get_slice(name=slice_name)
    for node in slice.get_nodes():
        print(f"Node: {node.get_ssh_command()}")
except Exception as e:
    print(f"Exception: {e}")

Node: ssh -i /home/fabric/work/id_rsa -J cqy78_0038438951@bastion-1.fabric-testbed.net ubuntu@2607:f018:110:11:f816:3eff:fe8f:6dc0
Node: ssh -i /home/fabric/work/id_rsa -J cqy78_0038438951@bastion-1.fabric-testbed.net ubuntu@2607:f018:110:11:f816:3eff:fe1b:218c
Node: ssh -i /home/fabric/work/id_rsa -J cqy78_0038438951@bastion-1.fabric-testbed.net ubuntu@2001:400:a100:3030:f816:3eff:fee1:cc0c
Node: ssh -i /home/fabric/work/id_rsa -J cqy78_0038438951@bastion-1.fabric-testbed.net ubuntu@2001:400:a100:3030:f816:3eff:fe24:dceb


## Configure Nodes
 ### This is just an exmple. Please wait for github to turn public and then run the script. Or the system will wait for cerdential.
 ### STEP 1: Setup controll net ---NO NEED

 ### STEP 2: Setup LAN net

In [12]:
try:
    h1 = slice.get_node(name=h1_name)        
    h1_os_iface = h1.get_interface(network_name=prod_lan_net_name)
    h1_os_iface.set_ip(ip=h1_LAN_ip, cidr="24")
    
    print(f"ifname: {h1_os_iface.get_os_interface()}")
    print(f"mac: {h1_os_iface.get_mac()}")
    stdout, stderr = h1.execute(f'sudo ip addr list {h1_os_iface.get_os_interface()}')
    print (stdout)
    print (stderr)
    
    #stdout, stderr = h1.execute(host_config_script_h1)
    #print("stdout: {}".format(stdout))
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()
    
try:
    h2 = slice.get_node(name=h2_name)
    h2_os_iface = h2.get_interface(network_name=prod_lan_net_name)
    h2_os_iface.set_ip(ip=h2_LAN_ip, cidr="24")
 
    print(f"ifname: {h2_os_iface.get_os_interface()}")
    print(f"mac: {h2_os_iface.get_mac()}")
    stdout, stderr = h2.execute(f'sudo ip addr list {h2_os_iface.get_os_interface()}')
    print (stdout)
    print (stderr)
    
    #stdout, stderr = h2.execute(host_config_script_h2)
    #print("stdout: {}".format(stdout))
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()
    

try:
    h4 = slice.get_node(name=h4_name)
    h4_os_iface = h4.get_interface(network_name=cons_lan_net_name)
    h4_os_iface.set_ip(ip=h4_LAN_ip, cidr="24")

    print(f"ifname: {h4_os_iface.get_os_interface()}")
    print(f"mac: {h4_os_iface.get_mac()}")
    stdout, stderr = h4.execute(f'sudo ip addr list {h4_os_iface.get_os_interface()}')
    print (stdout)
    print (stderr)
    
    #stdout, stderr = h3.execute(host_config_script_h3)
    #print("stdout: {}".format(stdout))
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()
    
try:
    h5 = slice.get_node(name=h5_name)
    h5_os_iface = h5.get_interface(network_name=cons_lan_net_name)
    h5_os_iface.set_ip(ip=h5_LAN_ip, cidr="24")

    print(f"ifname: {h5_os_iface.get_os_interface()}")
    print(f"mac: {h5_os_iface.get_mac()}")
    stdout, stderr = h5.execute(f'sudo ip addr list {h5_os_iface.get_os_interface()}')
    print (stdout)
    print (stderr)
    
    #stdout, stderr = h3.execute(host_config_script_h3)
    #print("stdout: {}".format(stdout))
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()

ifname: ens7
mac: 02:B3:F5:DB:97:7C
3: ens7: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc mq state UP group default qlen 1000
    link/ether 02:b3:f5:db:97:7c brd ff:ff:ff:ff:ff:ff
    inet 172.16.0.1/24 scope global ens7
       valid_lft forever preferred_lft forever
    inet6 fe80::b3:f5ff:fedb:977c/64 scope link 
       valid_lft forever preferred_lft forever


ifname: ens7
mac: 06:1D:A4:2D:F2:99
3: ens7: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc mq state UP group default qlen 1000
    link/ether 06:1d:a4:2d:f2:99 brd ff:ff:ff:ff:ff:ff
    inet 172.16.0.2/24 scope global ens7
       valid_lft forever preferred_lft forever
    inet6 fe80::41d:a4ff:fe2d:f299/64 scope link 
       valid_lft forever preferred_lft forever


ifname: ens8
mac: 0A:28:B5:6D:ED:AC
4: ens8: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc mq state UP group default qlen 1000
    link/ether 0a:28:b5:6d:ed:ac brd ff:ff:ff:ff:ff:ff
    inet 172.16.2.1/24 scope global ens8
       valid_lft forever prefer

 ### STEP 3: Setup WAN net

In [13]:
try:
    h2 = slice.get_node(name=h2_name)
    h2_os_iface = h2.get_interface(network_name=gn_net_name)
    h2_os_iface.set_ip(ip=h2_WAN_ip, cidr="24")
 
    print(f"ifname: {h2_os_iface.get_os_interface()}")
    print(f"mac: {h2_os_iface.get_mac()}")
    stdout, stderr = h2.execute(f'sudo ip addr list {h2_os_iface.get_os_interface()}')
    print (stdout)
    print (stderr)
    
    #stdout, stderr = h2.execute(host_config_script_h2)
    #print("stdout: {}".format(stdout))
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()
    

try:
    h4 = slice.get_node(name=h4_name)
    h4_os_iface = h4.get_interface(network_name=gn_net_name)
    h4_os_iface.set_ip(ip=h4_WAN_ip, cidr="24")

    print(f"ifname: {h4_os_iface.get_os_interface()}")
    print(f"mac: {h4_os_iface.get_mac()}")
    stdout, stderr = h4.execute(f'sudo ip addr list {h4_os_iface.get_os_interface()}')
    print (stdout)
    print (stderr)
    
    #stdout, stderr = h3.execute(host_config_script_h3)
    #print("stdout: {}".format(stdout))
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()

ifname: ens8
mac: 0A:44:2E:A3:7B:F8
4: ens8: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc mq state UP group default qlen 1000
    link/ether 0a:44:2e:a3:7b:f8 brd ff:ff:ff:ff:ff:ff
    inet 172.16.1.1/24 scope global ens8
       valid_lft forever preferred_lft forever
    inet6 fe80::844:2eff:fea3:7bf8/64 scope link 
       valid_lft forever preferred_lft forever


ifname: ens7
mac: 0A:07:62:7E:09:E1
3: ens7: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc mq state UP group default qlen 1000
    link/ether 0a:07:62:7e:09:e1 brd ff:ff:ff:ff:ff:ff
    inet 172.16.1.2/24 scope global ens7
       valid_lft forever preferred_lft forever
    inet6 fe80::807:62ff:fe7e:9e1/64 scope link 
       valid_lft forever preferred_lft forever




## TCP Tunning and iperf test

In [14]:
stdout, stderr = h1.execute(f'sudo apt update && sudo apt install -y iperf iperf3 net-tools')
stdout, stderr = h2.execute(f'sudo apt update && sudo apt install -y iperf iperf3 net-tools')

stdout, stderr = h4.execute(f'sudo apt update && sudo apt install -y iperf iperf3 net-tools')
stdout, stderr = h5.execute(f'sudo apt update && sudo apt install -y iperf iperf3 net-tools')

In [15]:
stdout, stderr = h1.execute('echo "net.core.rmem_max = 536870912\nnet.core.wmem_max = 536870912\nnet.ipv4.tcp_rmem = 4096 87380 536870912\nnet.ipv4.tcp_wmem = 4096 65536 536870912\nnet.ipv4.tcp_congestion_control=htcp\nnet.ipv4.tcp_mtu_probing=1\nnet.core.default_qdisc = fq\n" | sudo tee -a /etc/sysctl.conf && sudo sysctl -p')

stdout, stderr = h2.execute('echo "net.core.rmem_max = 536870912\nnet.core.wmem_max = 536870912\nnet.ipv4.tcp_rmem = 4096 87380 536870912\nnet.ipv4.tcp_wmem = 4096 65536 536870912\nnet.ipv4.tcp_congestion_control=htcp\nnet.ipv4.tcp_mtu_probing=1\nnet.core.default_qdisc = fq\n" | sudo tee -a /etc/sysctl.conf && sudo sysctl -p')


stdout, stderr = h4.execute('echo "net.core.rmem_max = 536870912\nnet.core.wmem_max = 536870912\nnet.ipv4.tcp_rmem = 4096 87380 536870912\nnet.ipv4.tcp_wmem = 4096 65536 536870912\nnet.ipv4.tcp_congestion_control=htcp\nnet.ipv4.tcp_mtu_probing=1\nnet.core.default_qdisc = fq\n" | sudo tee -a /etc/sysctl.conf && sudo sysctl -p')

stdout, stderr = h5.execute('echo "net.core.rmem_max = 536870912\nnet.core.wmem_max = 536870912\nnet.ipv4.tcp_rmem = 4096 87380 536870912\nnet.ipv4.tcp_wmem = 4096 65536 536870912\nnet.ipv4.tcp_congestion_control=htcp\nnet.ipv4.tcp_mtu_probing=1\nnet.core.default_qdisc = fq\n" | sudo tee -a /etc/sysctl.conf && sudo sysctl -p')


In [16]:
#ATTENTION: be aware of the NIC id!
#NEED SUDO! 
stdout, stderr = h1.execute('sudo ifconfig ens7 mtu 8900 up && sudo tc qdisc add dev ens7 root fq maxrate 30gbit')
print (stdout)
print (stderr)
stdout, stderr = h2.execute('sudo ifconfig ens7 mtu 8900 up && sudo tc qdisc add dev ens7 root fq maxrate 30gbit')
stdout, stderr = h2.execute('sudo ifconfig ens8 mtu 8900 up && sudo tc qdisc add dev ens8 root fq maxrate 30gbit')
print (stdout)
print (stderr)


stdout, stderr = h4.execute('sudo ifconfig ens7 mtu 8900 up && sudo tc qdisc add dev ens7 root fq maxrate 30gbit')
stdout, stderr = h4.execute('sudo ifconfig ens8 mtu 8900 up && sudo tc qdisc add dev ens8 root fq maxrate 30gbit')
print (stdout)
print (stderr)
stdout, stderr = h5.execute('sudo ifconfig ens7 mtu 8900 up && sudo tc qdisc add dev ens7 root fq maxrate 30gbit')
print (stdout)
print (stderr)

In [17]:

h2 = slice.get_node(name=h2_name) 
stdout, stderr = h2.execute('tmux new -d \'iperf -s -f K\'')
print (stdout)
print (stderr)

In [18]:
#TCP TEST:
#Forward direction:
h1 = slice.get_node(name=h1_name) 
stdout, stderr = h1.execute(f'iperf -c 172.16.0.2 -P 32 -w 999M -t 30') 
print (stdout)
print (stderr)

------------------------------------------------------------
Client connecting to 172.16.0.2, TCP port 5001
TCP window size: 1.00 GByte (WARNING: requested  999 MByte)
------------------------------------------------------------
[ 33] local 172.16.0.1 port 59976 connected with 172.16.0.2 port 5001
[ 32] local 172.16.0.1 port 59970 connected with 172.16.0.2 port 5001
[ 31] local 172.16.0.1 port 59968 connected with 172.16.0.2 port 5001
[ 30] local 172.16.0.1 port 59972 connected with 172.16.0.2 port 5001
[ 34] local 172.16.0.1 port 59974 connected with 172.16.0.2 port 5001
[  7] local 172.16.0.1 port 59922 connected with 172.16.0.2 port 5001
[ 13] local 172.16.0.1 port 59926 connected with 172.16.0.2 port 5001
[  4] local 172.16.0.1 port 59914 connected with 172.16.0.2 port 5001
[  3] local 172.16.0.1 port 59916 connected with 172.16.0.2 port 5001
[ 15] local 172.16.0.1 port 59928 connected with 172.16.0.2 port 5001
[  6] local 172.16.0.1 port 59920 connected with 172.16.0.2 port 5001
[

In [19]:
#Backward direction:
h4 = slice.get_node(name=h4_name) 
stdout, stderr = h4.execute(f'iperf -c 172.16.1.1 -P 32 -w 999M -t 30') 
print (stdout)
print (stderr)

------------------------------------------------------------
Client connecting to 172.16.1.1, TCP port 5001
TCP window size: 1.00 GByte (WARNING: requested  999 MByte)
------------------------------------------------------------
[ 30] local 172.16.1.2 port 53822 connected with 172.16.1.1 port 5001
[ 13] local 172.16.1.2 port 53788 connected with 172.16.1.1 port 5001
[  8] local 172.16.1.2 port 53780 connected with 172.16.1.1 port 5001
[ 24] local 172.16.1.2 port 53808 connected with 172.16.1.1 port 5001
[ 21] local 172.16.1.2 port 53804 connected with 172.16.1.1 port 5001
[ 25] local 172.16.1.2 port 53812 connected with 172.16.1.1 port 5001
[  3] local 172.16.1.2 port 53770 connected with 172.16.1.1 port 5001
[  6] local 172.16.1.2 port 53774 connected with 172.16.1.1 port 5001
[ 18] local 172.16.1.2 port 53798 connected with 172.16.1.1 port 5001
[ 26] local 172.16.1.2 port 53814 connected with 172.16.1.1 port 5001
[ 17] local 172.16.1.2 port 53796 connected with 172.16.1.1 port 5001
[

In [20]:
#UDP TEST:
#Forward direction
h1 = slice.get_node(name=h1_name) 
stdout, stderr = h1.execute(f'iperf -l8848 -T30 -u -w4m -b10G -c 172.16.0.2 -P 4') 
print (stdout)
print (stderr)

------------------------------------------------------------
Client connecting to 172.16.0.2, UDP port 5001
Sending 8848 byte datagrams, IPG target: 6.59 us (kalman adjust)
UDP buffer size: 7.63 MByte (WARNING: requested 3.81 MByte)
------------------------------------------------------------
[  6] local 172.16.0.1 port 43458 connected with 172.16.0.2 port 5001
[  3] local 172.16.0.1 port 60070 connected with 172.16.0.2 port 5001
[  4] local 172.16.0.1 port 49608 connected with 172.16.0.2 port 5001
[  5] local 172.16.0.1 port 34777 connected with 172.16.0.2 port 5001
[ ID] Interval       Transfer     Bandwidth
[  3]  0.0-10.0 sec  10.3 GBytes  8.89 Gbits/sec
[  3] Sent 1255954 datagrams
[  4]  0.0-10.0 sec  9.44 GBytes  8.11 Gbits/sec
[  4] Sent 1146165 datagrams
[  6]  0.0-10.0 sec  10.1 GBytes  8.68 Gbits/sec
[  6] Sent 1226091 datagrams
[  5]  0.0-10.0 sec  7.80 GBytes  6.70 Gbits/sec
[  5] Sent 946936 datagrams
[SUM]  0.0-10.0 sec  37.7 GBytes  32.4 Gbits/sec
[SUM] Sent 4575146 dat

In [21]:
#Backward direction
h4 = slice.get_node(name=h4_name) 
stdout, stderr = h4.execute(f'iperf -l8848 -T30 -u -w4m -b10G -c 172.16.1.1 -P 4') 
print (stdout)
print (stderr)

------------------------------------------------------------
Client connecting to 172.16.1.1, UDP port 5001
Sending 8848 byte datagrams, IPG target: 6.59 us (kalman adjust)
UDP buffer size: 7.63 MByte (WARNING: requested 3.81 MByte)
------------------------------------------------------------
[  6] local 172.16.1.2 port 35416 connected with 172.16.1.1 port 5001
[  3] local 172.16.1.2 port 46772 connected with 172.16.1.1 port 5001
[  4] local 172.16.1.2 port 55543 connected with 172.16.1.1 port 5001
[  5] local 172.16.1.2 port 52036 connected with 172.16.1.1 port 5001
[ ID] Interval       Transfer     Bandwidth
[  3]  0.0-10.0 sec  5.36 GBytes  4.61 Gbits/sec
[  3] Sent 650998 datagrams
[  5]  0.0-10.0 sec  9.05 GBytes  7.77 Gbits/sec
[  5] Sent 1098221 datagrams
[  6]  0.0-10.0 sec  9.44 GBytes  8.11 Gbits/sec
[  6] Sent 1145149 datagrams
[  4]  0.0-10.0 sec  4.09 GBytes  3.52 Gbits/sec
[  4] Sent 496593 datagrams
[SUM]  0.0-10.0 sec  27.9 GBytes  24.0 Gbits/sec
[SUM] Sent 3390961 data

In [22]:
h4 = slice.get_node(name=h4_name) 
stdout, stderr = h4.execute('tmux new -d \'iperf -s -f K\'')
print (stdout)
print (stderr)

In [23]:
#TCP TEST:
#Forward direction
h2 = slice.get_node(name=h2_name) 
stdout, stderr = h2.execute(f'iperf -c 172.16.1.2 -P 32 -w 999M') 
print (stdout)
print (stderr)

------------------------------------------------------------
Client connecting to 172.16.1.2, TCP port 5001
TCP window size: 1.00 GByte (WARNING: requested  999 MByte)
------------------------------------------------------------
[ 34] local 172.16.1.1 port 53130 connected with 172.16.1.2 port 5001
[ 28] local 172.16.1.1 port 53118 connected with 172.16.1.2 port 5001
[ 31] local 172.16.1.1 port 53124 connected with 172.16.1.2 port 5001
[ 33] local 172.16.1.1 port 53128 connected with 172.16.1.2 port 5001
[ 32] local 172.16.1.1 port 53126 connected with 172.16.1.2 port 5001
[  3] local 172.16.1.1 port 53070 connected with 172.16.1.2 port 5001
[  6] local 172.16.1.1 port 53074 connected with 172.16.1.2 port 5001
[  7] local 172.16.1.1 port 53076 connected with 172.16.1.2 port 5001
[  5] local 172.16.1.1 port 53072 connected with 172.16.1.2 port 5001
[  8] local 172.16.1.1 port 53080 connected with 172.16.1.2 port 5001
[  4] local 172.16.1.1 port 53068 connected with 172.16.1.2 port 5001
[

In [24]:
#Backward direction
h5 = slice.get_node(name=h5_name) 
stdout, stderr = h5.execute(f'iperf -c 172.16.2.1 -P 32 -w 999M') 
print (stdout)
print (stderr)

------------------------------------------------------------
Client connecting to 172.16.2.1, TCP port 5001
TCP window size: 1.00 GByte (WARNING: requested  999 MByte)
------------------------------------------------------------
[ 32] local 172.16.2.2 port 48424 connected with 172.16.2.1 port 5001
[ 30] local 172.16.2.2 port 48418 connected with 172.16.2.1 port 5001
[ 31] local 172.16.2.2 port 48420 connected with 172.16.2.1 port 5001
[ 34] local 172.16.2.2 port 48422 connected with 172.16.2.1 port 5001
[ 35] local 172.16.2.2 port 48416 connected with 172.16.2.1 port 5001
[  4] local 172.16.2.2 port 48362 connected with 172.16.2.1 port 5001
[  5] local 172.16.2.2 port 48364 connected with 172.16.2.1 port 5001
[  3] local 172.16.2.2 port 48360 connected with 172.16.2.1 port 5001
[  6] local 172.16.2.2 port 48366 connected with 172.16.2.1 port 5001
[  9] local 172.16.2.2 port 48372 connected with 172.16.2.1 port 5001
[ 12] local 172.16.2.2 port 48380 connected with 172.16.2.1 port 5001
[

In [25]:
#UDP TEST:
#Forward direction
h2 = slice.get_node(name=h2_name) 
stdout, stderr = h2.execute(f'iperf -l8848 -T30 -u -w4m -b10G -c 172.16.1.2 -P 4') 
print (stdout)
print (stderr)

------------------------------------------------------------
Client connecting to 172.16.1.2, UDP port 5001
Sending 8848 byte datagrams, IPG target: 6.59 us (kalman adjust)
UDP buffer size: 7.63 MByte (WARNING: requested 3.81 MByte)
------------------------------------------------------------
[  6] local 172.16.1.1 port 50444 connected with 172.16.1.2 port 5001
[  3] local 172.16.1.1 port 59865 connected with 172.16.1.2 port 5001
[  4] local 172.16.1.1 port 37616 connected with 172.16.1.2 port 5001
[  5] local 172.16.1.1 port 44921 connected with 172.16.1.2 port 5001
[ ID] Interval       Transfer     Bandwidth
[  3]  0.0-10.0 sec  9.42 GBytes  8.10 Gbits/sec
[  3] Sent 1143756 datagrams
[  5]  0.0-10.0 sec  9.13 GBytes  7.84 Gbits/sec
[  5] Sent 1107535 datagrams
[  4]  0.0-10.0 sec  7.15 GBytes  6.14 Gbits/sec
[  4] Sent 867270 datagrams
[  6]  0.0-10.0 sec  3.07 GBytes  2.63 Gbits/sec
[  6] Sent 372056 datagrams
[SUM]  0.0-10.0 sec  28.8 GBytes  24.7 Gbits/sec
[SUM] Sent 3490617 data

In [26]:
#Backward direction
h5 = slice.get_node(name=h5_name) 
stdout, stderr = h5.execute(f'iperf -l8848 -T30 -u -w4m -b10G -c 172.16.2.1 -P 4') 
print (stdout)
print (stderr)

------------------------------------------------------------
Client connecting to 172.16.2.1, UDP port 5001
Sending 8848 byte datagrams, IPG target: 6.59 us (kalman adjust)
UDP buffer size: 7.63 MByte (WARNING: requested 3.81 MByte)
------------------------------------------------------------
[  5] local 172.16.2.2 port 40147 connected with 172.16.2.1 port 5001
[  3] local 172.16.2.2 port 50361 connected with 172.16.2.1 port 5001
[  4] local 172.16.2.2 port 40549 connected with 172.16.2.1 port 5001
[  6] local 172.16.2.2 port 52782 connected with 172.16.2.1 port 5001
[ ID] Interval       Transfer     Bandwidth
[  5]  0.0-10.0 sec  8.47 GBytes  7.27 Gbits/sec
[  5] Sent 1027284 datagrams
[  3]  0.0-10.0 sec  8.09 GBytes  6.95 Gbits/sec
[  3] Sent 982078 datagrams
[  4]  0.0-10.0 sec  7.47 GBytes  6.42 Gbits/sec
[  4] Sent 906433 datagrams
[  6]  0.0-10.0 sec  2.64 GBytes  2.27 Gbits/sec
[  6] Sent 320550 datagrams
[SUM]  0.0-10.0 sec  26.7 GBytes  22.9 Gbits/sec
[SUM] Sent 3236345 datag

In [5]:
h1 = slice.get_node(name=h1_name) 
stdout, stderr = h1.execute('tmux new -d \'iperf -s -f K\'')
print (stdout)
print (stderr)

In [6]:
#TCP TEST:
#Backward direction
h2 = slice.get_node(name=h2_name) 
stdout, stderr = h2.execute(f'iperf -c 172.16.0.1 -P 32 -w 999M') 
print (stdout)
print (stderr)

------------------------------------------------------------
Client connecting to 172.16.0.1, TCP port 5001
TCP window size: 1.00 GByte (WARNING: requested  999 MByte)
------------------------------------------------------------
[ 35] local 172.16.0.2 port 52736 connected with 172.16.0.1 port 5001
[ 33] local 172.16.0.2 port 52732 connected with 172.16.0.1 port 5001
[ 28] local 172.16.0.2 port 52730 connected with 172.16.0.1 port 5001
[ 32] local 172.16.0.2 port 52734 connected with 172.16.0.1 port 5001
[  4] local 172.16.0.2 port 52676 connected with 172.16.0.1 port 5001
[  5] local 172.16.0.2 port 52678 connected with 172.16.0.1 port 5001
[  7] local 172.16.0.2 port 52680 connected with 172.16.0.1 port 5001
[  3] local 172.16.0.2 port 52674 connected with 172.16.0.1 port 5001
[  6] local 172.16.0.2 port 52682 connected with 172.16.0.1 port 5001
[  8] local 172.16.0.2 port 52684 connected with 172.16.0.1 port 5001
[ 13] local 172.16.0.2 port 52690 connected with 172.16.0.1 port 5001
[

In [7]:
#UDP TEST:
#Backward direction
h2 = slice.get_node(name=h2_name) 
stdout, stderr = h2.execute(f'iperf -l8848 -T30 -u -w4m -b10G -c 172.16.0.1 -P 4') 
print (stdout)
print (stderr)

------------------------------------------------------------
Client connecting to 172.16.0.1, UDP port 5001
Sending 8848 byte datagrams, IPG target: 6.59 us (kalman adjust)
UDP buffer size: 7.63 MByte (WARNING: requested 3.81 MByte)
------------------------------------------------------------
[  6] local 172.16.0.2 port 42065 connected with 172.16.0.1 port 5001
[  3] local 172.16.0.2 port 45776 connected with 172.16.0.1 port 5001
[  4] local 172.16.0.2 port 43064 connected with 172.16.0.1 port 5001
[  5] local 172.16.0.2 port 50594 connected with 172.16.0.1 port 5001
[ ID] Interval       Transfer     Bandwidth
[  3]  0.0-10.0 sec  8.31 GBytes  7.13 Gbits/sec
[  3] Sent 1007985 datagrams
[  6]  0.0-10.0 sec  6.69 GBytes  5.75 Gbits/sec
[  6] Sent 811893 datagrams
[  4]  0.0-10.0 sec  6.98 GBytes  5.99 Gbits/sec
[  4] Sent 846673 datagrams
[  5]  0.0-10.0 sec  6.61 GBytes  5.68 Gbits/sec
[  5] Sent 802016 datagrams
[SUM]  0.0-10.0 sec  28.6 GBytes  24.6 Gbits/sec
[SUM] Sent 3468567 datag

In [8]:
h5 = slice.get_node(name=h5_name) 
stdout, stderr = h5.execute('tmux new -d \'iperf -s -f K\'')
print (stdout)
print (stderr)

In [9]:
#Backward direction:
h4 = slice.get_node(name=h4_name) 
stdout, stderr = h4.execute(f'iperf -c 172.16.2.2 -P 32 -w 999M -t 30') 
print (stdout)
print (stderr)

------------------------------------------------------------
Client connecting to 172.16.2.2, TCP port 5001
TCP window size: 1.00 GByte (WARNING: requested  999 MByte)
------------------------------------------------------------
[ 33] local 172.16.2.1 port 53714 connected with 172.16.2.2 port 5001
[ 31] local 172.16.2.1 port 53708 connected with 172.16.2.2 port 5001
[ 32] local 172.16.2.1 port 53710 connected with 172.16.2.2 port 5001
[ 34] local 172.16.2.1 port 53712 connected with 172.16.2.2 port 5001
[  6] local 172.16.2.1 port 53656 connected with 172.16.2.2 port 5001
[  4] local 172.16.2.1 port 53654 connected with 172.16.2.2 port 5001
[ 11] local 172.16.2.1 port 53660 connected with 172.16.2.2 port 5001
[  3] local 172.16.2.1 port 53650 connected with 172.16.2.2 port 5001
[  5] local 172.16.2.1 port 53652 connected with 172.16.2.2 port 5001
[ 12] local 172.16.2.1 port 53662 connected with 172.16.2.2 port 5001
[  7] local 172.16.2.1 port 53658 connected with 172.16.2.2 port 5001
[

In [10]:
#Forward direction
h4 = slice.get_node(name=h4_name) 
stdout, stderr = h4.execute(f'iperf -l8848 -T30 -u -w4m -b10G -c 172.16.2.2 -P 4') 
print (stdout)
print (stderr)

------------------------------------------------------------
Client connecting to 172.16.2.2, UDP port 5001
Sending 8848 byte datagrams, IPG target: 6.59 us (kalman adjust)
UDP buffer size: 7.63 MByte (WARNING: requested 3.81 MByte)
------------------------------------------------------------
[  6] local 172.16.2.1 port 42941 connected with 172.16.2.2 port 5001
[  3] local 172.16.2.1 port 34730 connected with 172.16.2.2 port 5001
[  4] local 172.16.2.1 port 57773 connected with 172.16.2.2 port 5001
[  5] local 172.16.2.1 port 40184 connected with 172.16.2.2 port 5001
[ ID] Interval       Transfer     Bandwidth
[  4]  0.0-10.0 sec  8.88 GBytes  7.63 Gbits/sec
[  4] Sent 1077931 datagrams
[  3]  0.0-10.0 sec  8.86 GBytes  7.61 Gbits/sec
[  3] Sent 1074663 datagrams
[  6]  0.0-10.0 sec  7.12 GBytes  6.12 Gbits/sec
[  6] Sent 864248 datagrams
[  5]  0.0-10.0 sec  2.05 GBytes  1.76 Gbits/sec
[  5] Sent 248355 datagrams
[SUM]  0.0-10.0 sec  26.9 GBytes  23.1 Gbits/sec
[SUM] Sent 3265197 data

# Upload File and Execute

In [27]:
#upload_file(local_file_path, remote_file_path, retry=3, retry_interval=10)
# FILE 1: scistream source code

try:
    h1 = slice.get_node(name=h1_name)
    h1.upload_file('../scistream-source-code.zip',"scistream-source-code.zip", retry=3, retry_interval=10)
    
    h2 = slice.get_node(name=h2_name)
    h2.upload_file('../scistream-source-code.zip',"scistream-source-code.zip", retry=3, retry_interval=10)
    
    h4 = slice.get_node(name=h4_name)
    h4.upload_file('../scistream-source-code.zip',"scistream-source-code.zip", retry=3, retry_interval=10)
    h5 = slice.get_node(name=h5_name)
    h5.upload_file('../scistream-source-code.zip',"scistream-source-code.zip", retry=3, retry_interval=10)
    
    #stdout, stderr = h3.execute(host_config_script_h3)
    #print("stdout: {}".format(stdout))
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()

In [28]:
#FILE 2: setup.sh
try:
    h1 = slice.get_node(name=h1_name)
    h1.upload_file('../setup.sh',"setup.sh", retry=3, retry_interval=10)
    
    h2 = slice.get_node(name=h2_name)
    h2.upload_file('../setup.sh',"setup.sh", retry=3, retry_interval=10)
   
    h4 = slice.get_node(name=h4_name)
    h4.upload_file('../setup.sh',"setup.sh", retry=3, retry_interval=10)
    h5 = slice.get_node(name=h5_name)
    h5.upload_file('../setup.sh',"setup.sh", retry=3, retry_interval=10)
    
    #stdout, stderr = h3.execute(host_config_script_h3)
    #print("stdout: {}".format(stdout))
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()

In [29]:
#Optional file: QUIC proxy exmaple
try:
    h1 = slice.get_node(name=h1_name)
    h1.upload_file('../quic-tun_0.0.1_linux_amd64.tar.gz',"quic-tun_0.0.1_linux_amd64.tar.gz", retry=3, retry_interval=10)
    
    h2 = slice.get_node(name=h2_name)
    h2.upload_file('../quic-tun_0.0.1_linux_amd64.tar.gz',"quic-tun_0.0.1_linux_amd64.tar.gz", retry=3, retry_interval=10)
   
    h4 = slice.get_node(name=h4_name)
    h4.upload_file('../quic-tun_0.0.1_linux_amd64.tar.gz',"quic-tun_0.0.1_linux_amd64.tar.gz", retry=3, retry_interval=10)
    h5 = slice.get_node(name=h5_name)
    h5.upload_file('../quic-tun_0.0.1_linux_amd64.tar.gz',"quic-tun_0.0.1_linux_amd64.tar.gz", retry=3, retry_interval=10)
    
    #stdout, stderr = h3.execute(host_config_script_h3)
    #print("stdout: {}".format(stdout))
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()

In [30]:
#Optional file: QUIC tunnel example
try:
    h1 = slice.get_node(name=h1_name)
    h1.upload_file('../quic-proxy-example.zip',"quic-proxy-example.zip", retry=3, retry_interval=10)
    
    h2 = slice.get_node(name=h2_name)
    h2.upload_file('../quic-proxy-example.zip',"quic-proxy-example.zip", retry=3, retry_interval=10)
    h3 = slice.get_node(name=h3_name)
    h3.upload_file('../quic-proxy-example.zip',"quic-proxy-example.zip", retry=3, retry_interval=10)
    h4 = slice.get_node(name=h4_name)
    h4.upload_file('../quic-proxy-example.zip',"quic-proxy-example.zip", retry=3, retry_interval=10)
    h5 = slice.get_node(name=h5_name)
    h5.upload_file('../quic-proxy-example.zip',"quic-proxy-example.zip", retry=3, retry_interval=10)
    
    #stdout, stderr = h3.execute(host_config_script_h3)
    #print("stdout: {}".format(stdout))
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()

Error: Node not found: h3


Traceback (most recent call last):
  File "/opt/conda/lib/python3.9/site-packages/fabrictestbed_extensions/fablib/slice.py", line 739, in get_node
    return Node.get_node(self, self.get_fim_topology().nodes[name])
  File "/opt/conda/lib/python3.9/site-packages/fim/view_only_dict.py", line 19, in __getitem__
    return self.__dict.__getitem__(k)
KeyError: 'h3'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/tmp/ipykernel_53/2642736187.py", line 8, in <module>
    h3 = slice.get_node(name=h3_name)
  File "/opt/conda/lib/python3.9/site-packages/fabrictestbed_extensions/fablib/slice.py", line 742, in get_node
    raise Exception(f"Node not found: {name}")
Exception: Node not found: h3


In [31]:
#Optional file: quic distribuation - quic-go version...
try:
    slice = fablib.get_slice(name=slice_name)
    h1 = slice.get_node(name=h1_name)
    h1.upload_file('../quic-go.zip',"quic-go.zip", retry=3, retry_interval=10)
    
    h2 = slice.get_node(name=h2_name)
    h2.upload_file('../quic-go.zip',"quic-go.zip", retry=3, retry_interval=10)
    h3 = slice.get_node(name=h3_name)
    h3.upload_file('../quic-go.zip',"quic-go.zip", retry=3, retry_interval=10)
    h4 = slice.get_node(name=h4_name)
    h4.upload_file('../quic-go.zip',"quic-go.zip", retry=3, retry_interval=10)
    h5 = slice.get_node(name=h5_name)
    h5.upload_file('../quic-go.zip',"quic-go.zip", retry=3, retry_interval=10)
    
    #stdout, stderr = h3.execute(host_config_script_h3)
    #print("stdout: {}".format(stdout))
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()

Error: Node not found: h3


Traceback (most recent call last):
  File "/opt/conda/lib/python3.9/site-packages/fabrictestbed_extensions/fablib/slice.py", line 739, in get_node
    return Node.get_node(self, self.get_fim_topology().nodes[name])
  File "/opt/conda/lib/python3.9/site-packages/fim/view_only_dict.py", line 19, in __getitem__
    return self.__dict.__getitem__(k)
KeyError: 'h3'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/tmp/ipykernel_53/1409364764.py", line 9, in <module>
    h3 = slice.get_node(name=h3_name)
  File "/opt/conda/lib/python3.9/site-packages/fabrictestbed_extensions/fablib/slice.py", line 742, in get_node
    raise Exception(f"Node not found: {name}")
Exception: Node not found: h3


In [32]:
#FILE 3: mona demo, only upload on h1 and h5
# try:
#     slice = fablib.get_slice(name=slice_name)
#     h1 = slice.get_node(name=h1_name)
#     h1.upload_file('mona-demo.zip',"mona-demo.zip", retry=3, retry_interval=10)
    
#     h5 = slice.get_node(name=h5_name)
#     h5.upload_file('mona-demo.zip',"mona-demo.zip", retry=3, retry_interval=10)
    
#     #stdout, stderr = h3.execute(host_config_script_h3)
#     #print("stdout: {}".format(stdout))
# except Exception as e:
#     print(f"Error: {e}")
#     traceback.print_exc()

In [33]:
#Optional file: tls certs for public usage. Ignore if test-only 
# try:
#     slice = fablib.get_slice(name=slice_name)
#     h1 = slice.get_node(name=h1_name)
#     h1.upload_directory('newcerts/aioquic/newcerts',"newcerts", retry=3, retry_interval=10)
    
#     h2 = slice.get_node(name=h2_name)
#     h2.upload_directory('newcerts/aioquic/newcerts',"newcerts", retry=3, retry_interval=10)
#     h3 = slice.get_node(name=h3_name)
#     h3.upload_directory('newcerts/aioquic/newcerts',"newcerts", retry=3, retry_interval=10)
#     h4 = slice.get_node(name=h4_name)
#     h4.upload_directory('newcerts/aioquic/newcerts',"newcerts", retry=3, retry_interval=10)
#     h5 = slice.get_node(name=h5_name)
#     h5.upload_directory('newcerts/aioquic/newcerts',"newcerts", retry=3, retry_interval=10)
    
#     #stdout, stderr = h3.execute(host_config_script_h3)
#     #print("stdout: {}".format(stdout))
# except Exception as e:
#     print(f"Error: {e}")
#     traceback.print_exc()

In [34]:
#Optional download file: dumpfile...
# try:
#     slice = fablib.get_slice(name=slice_name)
#     h1 = slice.get_node(name=h1_name)
#     h1.download_file("dumpFile/prodGn_LAN.cap",'prodGn_LAN.cap', retry=3, retry_interval=10)
#     h2 = slice.get_node(name=h2_name)
#     h2.download_file("dumpFile/prodGn_WAN.cap",'prodGn_WAN.cap', retry=3, retry_interval=10)
#     h4 = slice.get_node(name=h4_name)
#     h4.download_file("dumpFile/consGn_WAN.cap",'consGn_WAN.cap', retry=3, retry_interval=10)
#     h5 = slice.get_node(name=h5_name)
#     h5.download_file("dumpFile/consGn_LAN.cap",'consGn_LAN.cap', retry=3, retry_interval=10)
#     h5 = slice.get_node(name=h5_name)
#     h5.download_file("dumpFile/ssl_log.txt",'aioquic/ssl_log.txt', retry=3, retry_interval=10)
# except Exception as e:
#     print(f"Error: {e}")
#     traceback.print_exc()

## Configure Nodes ----VERBOSE


In [35]:
master_script = "bash setup.sh"
try:
    h1 = slice.get_node(name=h1_name)        
    stdout, stderr = h1.execute(master_script)
    #print("stdout: {}".format(stdout))
    print("done h1...")
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()
    
try:
    h2 = slice.get_node(name=h2_name)
    stdout, stderr = h2.execute(master_script)
    #print("stdout: {}".format(stdout))
    print("done h2...")
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()
    
try:
    h4 = slice.get_node(name=h4_name) 
    stdout, stderr = h4.execute(master_script)
    #print("stdout: {}".format(stdout))
    print("done h4...")
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()
    
try:
    h5 = slice.get_node(name=h5_name) 
    stdout, stderr = h5.execute(master_script)
    #print("stdout: {}".format(stdout))
    print("done h5...")
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()

done h1...
done h2...
done h4...
done h5...


## Below are some optinal operations for the slice

### Extend the slice reservation time

In [36]:
# import datetime

# #Set end host to now plus 1 day
# end_date = (datetime.datetime.utcnow() + datetime.timedelta(days=14)).strftime("%Y-%m-%d %H:%M:%S")

# try:
#     slice = fablib.get_slice(name=slice_name)

#     slice.renew(end_date)
# except Exception as e:
#     print(f"Exception: {e}")

### Check New Lease End Date

In [37]:
# try:
#     slice = fablib.get_slice(name=slice_name)
#     print(f"Lease End         : {slice.get_lease_end()}")
       
# except Exception as e:
#     print(f"Exception: {e}")

### DANGER ZONE --------*-------- Delete Slice

In [38]:
# try:
#     slice = fablib.get_slice(name=slice_name)
#     slice.delete()
# except Exception as e:
#     print(f"Fail: {e}")
#     traceback.print_exc()